# REQUIREMENTS
- Replace the ```observation_source_value``` and ```observation_source_concept_id``` for all records with ```observation_source_value = HealthInsurance_InsuranceTypeUpdate (ID 43528428, from The Basics)``` with the ```observation_source_value``` and ```observation_source_concept_ids``` for records with ```observation_source_value = Insurance_InsuranceType (ID 1384450, from HCAU)```.


- Map the [HCAU] field values to the corresponding [The Basics] fields when replacing. If there are no values in the [HCAU] fields, set [The Basics] fields to NULL.

## Set up 

In [ ]:
# Load Libraries

library(plyr) 
library(dplyr)
library(tidyr)
library(reticulate)
library(crayon)

In [ ]:
pd = reticulate::import("pandas")

In [ ]:
PROJECT_ID = ''

In [ ]:
#read as csv the list of PIDS where data needs to be overwritten
AC70_pids <- read.csv("AC70_PIDs.csv")

In [ ]:
#removing the 'p' before the ids
for (p in 1:length(AC70_pids$PID)) {
    AC70_pids$pid[p] <- as.numeric(substring(AC70_pids$PID[p], 2))
}

## Obtaining dataframes to use in the SQL query
- ```obs_pids_notin_list``` is a dataframe of person_ids in ```AC70_pids``` that ***are not** in the observation table when observation_source_concept_id = 43528428. For these, we will replace the corresponding fields in the observation table with NULL--> see below ```overwrite_query1```


- ```obs_pids_in_list``` is a dataframe of person_ids in ```AC70_pids``` that ***are*** in the observation table when observation_source_concept_id = 43528428. For these, we will replace the corresponding fields in the observation table with hcau fields (observation_source_concept_id = 1384450)--> see below ```overwrite_query2```

In [ ]:
query1 = " 
SELECT * FROM `{}.observation` o WHERE o.observation_source_concept_id = 43528428" 
obs_overwrite <- pd$read_gbq(query1 , PROJECT_ID, dialect="standard")

In [ ]:
obs_pids_notin_list <- as.numeric(filter(AC70_pids, !(pid %in% obs_overwrite$person_id))[,2])
obs_pids_in_list <- as.numeric(filter(AC70_pids, pid %in% obs_overwrite$person_id))

In [ ]:
cat(crayon::magenta("This shows that none of person_ids in [AC70_pids] 
are in the observation table with observation_source_concept_id = 1384450 table).They are not in the hcau table either."))

# THIS IS THE QUERY THAT WILL UPDATE THE FIELDS TO HCAU FIELDS- 


In [ ]:

overwrite_query1 = sprintf("
UPDATE `{}.observation` 
    SET  observation_id = NULL,
         person_id = person_id,
         observation_concept_id = NULL,
         observation_date = NULL,
         observation_datetime = NULL,
         observation_type_concept_id = NULL,
         value_as_number = NULL,
         value_as_string = NULL,
         value_as_concept_id = NULL,
         qualifier_concept_id = NULL,
         unit_concept_id = NULL,
         provider_id = NULL,
         visit_occurrence_id = NULL,
         observation_source_value = NULL,
         observation_source_concept_id = NULL,
         unit_source_value = NULL,
         qualifier_source_value = NULL,
         value_source_concept_id = NULL,
         value_source_value = NULL,
         questionnaire_response_id = NULL
    WHERE observation_source_concept_id = 43528428
    AND person_id IN (%s)",obs_pids_notin_list)

update1_observation_table <- pd$read_gbq(overwrite_query1 , PROJECT_ID, dialect="standard")


In [ ]:
#This query gives an error because [obs_pids_in_list] is empty as said earlier. 
#This should not be a problem when curation loads the correct list of pids in [AC70_pids <- read.csv("AC70_PIDs.csv")]

overwrite_query2 = sprintf("
UPDATE `{}.observation` as o
    SET  o.observation_id = hcau.observation_id,
         o.person_id = o.person_id,
         o.observation_concept_id = hcau.observation_concept_id,
         o.observation_date = hcau.observation_date,
         o.observation_datetime = hcau.observation_datetime,
         o.observation_type_concept_id = hcau.observation_type_concept_id,
         o.value_as_number = hcau.value_as_number,
         o.value_as_string = hcau.value_as_string,
         o.value_as_concept_id = hcau.value_as_concept_id,
         o.qualifier_concept_id = hcau.qualifier_concept_id,
         o.unit_concept_id = hcau.unit_concept_id,
         o.provider_id = hcau.provider_id,
         o.visit_occurrence_id = hcau.visit_occurrence_id,
         o.observation_source_value = hcau.observation_source_value,
         o.observation_source_concept_id = hcau.observation_source_concept_id,
         o.unit_source_value = hcau.unit_source_value,
         o.qualifier_source_value = hcau.qualifier_source_value,
         o.value_source_concept_id = hcau.value_source_concept_id,
         o.value_source_value = hcau.value_source_value,
         o.questionnaire_response_id = hcau.questionnaire_response_id
    FROM (SELECT * FROM `{}.observation`h WHERE h.observation_source_concept_id = 1384450) as hcau 
    WHERE o.observation_source_concept_id = 43528428 AND o.person_id IN (%s)", obs_pids_in_list)

update2_observation_table <- pd$read_gbq(overwrite_query2 , PROJECT_ID, dialect="standard")
